In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import time
import sqlite3
import matplotlib.pyplot as plt

In [2]:
def epoch_timestring(epoch_time):
    return time.strftime('%H:%M:%S', time.localtime(int(epoch_time))) #%d%B%YT%H:%M:%S

def date_epoch(date_string):
    utc_time = dt.strptime(date_string, "%Y-%m-%d")
    epoch_start_time = utc_time.timestamp()
    return epoch_start_time + 900, epoch_start_time + 86400

In [3]:
def downsample(data, factor):
    np_data = np.asarray(data)
    np_sliced = np_data[::factor]

    return np_sliced

In [4]:
def pumpstartstop(np_data, downsample_f = 1):
    np_data = downsample(np_data, downsample_f)
    pump_start = []
    pump_stop = []
    pump_dict = {}
    last_ind,last_val = 0,0
    for i,val in enumerate(np.diff(np_data)):
        if val > 10:
            if last_val!=1:
                pump_start.append(i)
                pump_dict[i]=1
                last_val = 1
                last_ind = i
        if val < -10:
            if last_val == -1:
                pump_stop.remove(last_ind)
                pump_dict.pop(last_ind)
                pump_stop.append(i)
                pump_dict[i]=-1
                last_ind = i
            else:
                pump_stop.append(i)
                pump_dict[i]=-1
                last_ind = i
                last_val = -1
    return pump_start, pump_stop, pump_dict

In [5]:
# date format '2019-03-06', all for all available dates

def pump_stat(date_string, source, downsample_factor = 150):
    conn = sqlite3.connect('/home/sohamp/project_course_sem6/git/Dash Server/water_data.db')
    cursor = conn.cursor()

    if date_string == 'all':
        start_epoch, end_epoch = 0, 2000000000#date_epoch(date_string)
    else:
        start_epoch, end_epoch = date_epoch(date_string)
    time_list, current_flow = None, None
    if source == 'CWPS':
        cursor.execute(
            'SELECT timeEpoch, Current_Flow, Daily_Flow FROM cwps WHERE timeEpoch>{} AND timeEpoch<{}'
                        .format(start_epoch, end_epoch))
        np_flow_cwps = np.asarray(cursor.fetchall())
        current_flow = np_flow_cwps[:,1]
        time_list = list(map(epoch_timestring, list(np_flow_cwps[:, 0])))

    elif source == 'WSC_FW':
        cursor.execute(
            'SELECT timeEpoch, current_flow, daily_flow FROM wsc1_fwp WHERE timeEpoch>{} AND timeEpoch<{}'
                        .format(start_epoch, end_epoch))
        np_flow_wsc1_fw = np.asarray(cursor.fetchall())
        current_flow = np_flow_wsc1_fw[:,1]
        time_list = list(map(epoch_timestring, list(np_flow_wsc1_fw[:, 0])))

    elif source == 'WSC_RW':
        cursor.execute(
            'SELECT timeEpoch, current_flow, daily_flow FROM wsc1_rwp WHERE timeEpoch>{} AND timeEpoch<{}'
                        .format(start_epoch, end_epoch))
        np_flow_wsc1_rw = np.asarray(cursor.fetchall())
        current_flow = np_flow_wsc1_rw[:,1]*4
        # reason for multiplication by 4: The recycled water flows are lesser than that for fresh and cwps water
        # To measure changes in slope, this amplification makes task easier
        time_list = list(map(epoch_timestring, list(np_flow_wsc1_rw[:, 0])))

    else: #error
        print("Incoorrect source, enter CWPS, WSC_FW, WSC_RW")
        return -1
    
    pump_start, pump_stop, pump_dict = pumpstartstop(current_flow, downsample_factor)
    downsampled_time = downsample(time_list, downsample_factor)

    pump_action = pd.DataFrame(columns=['source','date','time', 'action'])

    action_ = {1:'start', -1: 'stop'}
    for k in pump_dict:
        pump_action = pump_action.append({'source':source,'date': date_string,'time':downsampled_time[k], 'action': action_[pump_dict[k]]}, ignore_index=True)

    return pump_action

In [6]:
for dd in range(3,9):
    d_string = '2019-03-0{}'.format(dd)
    cwps = pump_stat(d_string, 'CWPS', 150)
    wsc_fw = pump_stat(d_string, 'WSC_FW', 150)
    wsc_rw = pump_stat(d_string, 'WSC_RW', 150)

    try:
        np_cwps = np.vstack([np_cwps, cwps.values])
        np_wsc_fw = np.vstack([np_wsc_fw, cwps.values])
        np_wsc_rw = np.vstack([np_wsc_rw, cwps.values])
    except NameError:
        np_cwps = cwps.values
        np_wsc_fw = wsc_fw.values
        np_wsc_rw = wsc_rw.values

In [33]:
cwps_start = np_wsc_fw[np_wsc_fw[:,3]=='start'][:,2]
cwps_stop = np_wsc_fw[np_wsc_fw[:,3]=='stop'][:,2]

In [35]:
cwps_stop = np.delete(cwps_stop,0,0)

In [40]:
cwps_start

array(['07:02:30', '12:00:28', '13:01:16', '15:57:53', '18:29:45',
       '22:51:19', '13:31:20', '17:10:13', '19:11:50', '22:14:07',
       '13:37:20', '17:10:07', '19:23:51', '13:37:32', '16:58:14',
       '19:11:56', '13:43:32', '16:58:15', '19:12:03', '21:56:09',
       '13:37:39'], dtype=object)

In [36]:
cwps_start[[0,3,7,10,13]], cwps_start[[1,4,8,11,14]], cwps_start[[2,5,9,12,15]], cwps_start[[6,16]]

cwps_stop[[0,3,7,10,13]], cwps_stop[[1,4,8,11,14]], cwps_stop[[2,5,9,12,15]], cwps_stop[[6,16]]

(array(['10:04:49', '18:23:40', '19:30:02', '19:05:36', '18:59:47'],
       dtype=object),
 array(['12:55:11', '18:41:54', '23:51:27', '19:48:13', '19:36:16'],
       dtype=object),
 array(['13:31:42', '14:13:48', '14:19:55', '14:26:12', '14:26:05'],
       dtype=object),
 array(['19:05:46', '18:47:46'], dtype=object))

In [37]:
def f(x):
#     return dt.strptime(x,"%H:%M:%S")
    return x[:-3]

cwps_start_dt = np.array([f(xi) for xi in cwps_start])
cwps_stop_dt = np.array([f(xi) for xi in cwps_stop])

In [38]:
cwps_start_dt[[0,3,7,10,13]], cwps_start_dt[[1,4,8,11,14]], cwps_start_dt[[2,5,9,12,15]], cwps_start_dt[[6,16]]

(array(['07:02', '15:57', '17:10', '13:37', '13:37'], dtype='<U5'),
 array(['12:00', '18:29', '19:11', '17:10', '16:58'], dtype='<U5'),
 array(['13:01', '22:51', '22:14', '19:23', '19:11'], dtype='<U5'),
 array(['13:31', '13:43'], dtype='<U5'))

In [39]:
cwps_stop_dt[[0,3,7,10,13]], cwps_stop_dt[[1,4,8,11,14]], cwps_stop_dt[[2,5,9,12,15]], cwps_stop_dt[[6,16]]

(array(['10:04', '18:23', '19:30', '19:05', '18:59'], dtype='<U5'),
 array(['12:55', '18:41', '23:51', '19:48', '19:36'], dtype='<U5'),
 array(['13:31', '14:13', '14:19', '14:26', '14:26'], dtype='<U5'),
 array(['19:05', '18:47'], dtype='<U5'))

#### Trial run code for debugging

In [16]:
downsample_factor = 150
conn = sqlite3.connect('/home/sohamp/project_course_sem6/git/Dash Server/water_data.db')
cursor = conn.cursor()
start_epoch, end_epoch = date_epoch(d_string)
cursor.execute(
    'SELECT timeEpoch, current_flow, daily_flow FROM wsc1_rwp WHERE timeEpoch>{} AND timeEpoch<{}'
                .format(start_epoch, end_epoch))
np_flow_wsc1_rw = np.asarray(cursor.fetchall())
current_flow = np_flow_wsc1_rw[:,1]*5
time_list = list(map(epoch_timestring, list(np_flow_wsc1_rw[:, 0])))

pump_start, pump_stop, pump_dict = pumpstartstop(current_flow, downsample_factor)
downsampled_time = downsample(time_list, downsample_factor)

pump_action = pd.DataFrame(columns=['time', 'action'])

action_ = {1:'start', -1: 'stop'}
for k in pump_dict:
    pump_action = pump_action.append({'time':downsampled_time[k], 'action': action_[pump_dict[k]]}, ignore_index=True)
